In [1]:

!pip install datasets==3.1.0
!pip install transformers==4.45.2 sentence-transformers==2.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [2]:
import csv
import gzip
import logging
import math
import os
from datetime import datetime

import torch
from torch.utils.data import DataLoader

from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEF1Evaluator, CESoftmaxAccuracyEvaluator
from sentence_transformers.evaluation import SequentialEvaluator
from sentence_transformers.readers import InputExample

import numpy as np

from sentence_transformers import InputExample
from tqdm.autonotebook import tqdm
from sentence_transformers.evaluation import SentenceEvaluator




A Cross-Encoder is a type of neural network used for sentence pair classification tasks, such as:
- Text similarity scoring
- Reranking search results
- Natural Language Inference (NLI)

Unlike Bi-Encoders, which encode each input separately, Cross-Encoders process both input texts together, allowing deeper interaction between them.<br>

What is "BAAI/bge-reranker-base"? <br>

This is a pretrained model from BAAI (Beijing Academy of Artificial Intelligence), specifically designed for reranking tasks.


In [3]:
model = CrossEncoder("BAAI/bge-reranker-base")

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Download the question answer data file.

In [4]:
!wget https://raw.githubusercontent.com/reza-rahim/published/refs/heads/main/Fine_tuningEmbeddings/data/generated_qa_pairs_detail.csv

--2025-02-03 04:01:23--  https://raw.githubusercontent.com/reza-rahim/published/refs/heads/main/Fine_tuningEmbeddings/data/generated_qa_pairs_detail.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80469 (79K) [text/plain]
Saving to: ‘generated_qa_pairs_detail.csv’

generated_qa_pairs_ 100%[===================>]  78.58K  --.-KB/s    in 0.02s   

2025-02-03 04:01:23 (4.17 MB/s) - ‘generated_qa_pairs_detail.csv’ saved [80469/80469]



Reads a CSV file containing questions and answers.
Extracts "Question" and "Answer" from each row.
Stores them in a dictionary with numeric string keys.
The dictionary qa_dict can later be used for quick lookups of Q&A pairs.


In [7]:
# Initialize an index counter for question-answer pairs
ind = 0

# Create an empty dictionary to store question-answer pairs
qa_dict = {}

# Open the CSV file in read mode
with open('generated_qa_pairs_detail.csv', mode='r') as file:
    # Read the CSV file as a dictionary where column names are the keys
    csv_reader = csv.DictReader(file)

    # Iterate through each row in the CSV file
    for row in csv_reader:
        # Extract the "Question" and "Answer" fields and store them in a dictionary
        data = {"q": row['Question'], "a": row['Answer']}

        # Store the question-answer pair in qa_dict using the index as a string key
        qa_dict[str(ind)] = data

        # Increment the index for the next entry
        ind += 1


Creates a mix of entailment and contradiction pairs to train an NLI model.
Helps the model distinguish real question-answer pairs from unrelated ones.
Balances the dataset by ensuring both entailment and contradiction labels exist.

In [9]:
import random

import numpy as np

label2int = {"contradiction": 0, "entailment": 1}

train_samples = []
# Initialize an empty list to store training samples
train_samples = []

# Iterate over each question-answer pair in the dictionary
for key in qa_dict:

    # Extract and clean the question and answer by removing double quotes
    q = qa_dict[key]['q'].replace("\"", "")
    a = qa_dict[key]['a'].replace("\"", "")

    # Add a correct (entailment) question-answer pair to the training set
    train_samples.append(InputExample(texts=[q, a], label=label2int["entailment"]))

    # If the current key index is in the first half of the dataset
    if int(key) < ind / 2:
        # Select a random answer from the second half of the dataset
        random_number = random.randint(ind // 2, ind - 1)
        n = qa_dict[str(random_number)]['a'].replace("\"", "")
        # Treat this as another valid (entailment) pair
        train_samples.append(InputExample(texts=[q, n], label=label2int["entailment"]))

    # If the current key index is in the second half of the dataset
    else:
        # Select a random answer from the first half of the dataset
        random_number = random.randint(0, (ind // 2) - 1)
        n = qa_dict[str(random_number)]['a'].replace("\"", "")
        # Treat this as a contradiction (incorrect question-answer pair)
        train_samples.append(InputExample(texts=[q, n], label=label2int["contradiction"]))


In [10]:
print(train_samples[0])

<InputExample> label: 1, texts: What is the publication date of the NVIDIA Corporation Annual Report 2024?;  The publication date of the NVIDIA Corporation Annual Report 2024 is February 21st, 2024.



# Define a custom evaluator class for Mean Squared Error (MSE) accuracy evaluation

In [11]:
from __future__ import annotations


# Import logging and set up a logger for debugging and tracking execution
logger = logging.getLogger(__name__)


# Define a custom evaluator class for Mean Squared Error (MSE) accuracy evaluation
class MSEAccuracyEvaluator(SentenceEvaluator):

    def __init__(self, dataloader: DataLoader, name: str = "", write_csv: bool = True, show_progress_bar: bool = True):
        """
        Initializes the evaluator.

        Parameters:
        - dataloader: DataLoader object containing the evaluation dataset.
        - name: (Optional) Name of the dataset (used for logging and file naming).
        - write_csv: Whether to write evaluation results to a CSV file.
        - show_progress_bar: Whether to display a progress bar during evaluation.
        """

        self.dataloader = dataloader
        self.name = name
        self.show_progress_bar = show_progress_bar

        # Define the CSV filename for storing evaluation results
        self.csv_file = "AccuracyEvaluator" + ("_" + name if name else "") + "_results.csv"
        self.csv_headers = ["epoch", "steps", "Accuracy"]
        self.write_csv = write_csv

    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) -> float:
        """
        Runs the evaluation on the given model.

        Parameters:
        - model: The model to be evaluated.
        - output_path: (Optional) Directory to save evaluation results.
        - epoch: The current training epoch.
        - steps: The number of steps completed in the current epoch.

        Returns:
        - mse: The computed Mean Squared Error (MSE) score.
        """

        # Set the model to evaluation mode (disables dropout layers, etc.)
        model.model.eval()
        total = 0  # Counter for the number of batches
        loss_total = 0  # Accumulator for total loss

        # Determine the log message format based on epoch and step information
        if epoch != -1:
            if steps == -1:
                out_txt = f" after epoch {epoch}:"
            else:
                out_txt = f" in epoch {epoch} after {steps} steps:"
        else:
            out_txt = ":"

        # Log the start of evaluation
        logger.info("AccuracyEvaluator: Evaluating the model on " + self.name + " dataset" + out_txt)

        # Define the loss function (Mean Squared Error) and activation function (Sigmoid)
        loss_fnc = torch.nn.MSELoss()
        activation_fnc = torch.nn.Sigmoid()

        # Use the model's smart batching function to handle input data batching
        self.dataloader.collate_fn = model.smart_batching_collate

        logger.info("Evaluation on the " + self.name + " dataset" + out_txt)

        # Iterate through the dataset and evaluate the model
        for features, labels in tqdm(self.dataloader, desc="Evaluation", smoothing=0.05, disable=not self.show_progress_bar):
            with torch.no_grad():  # Disable gradient calculation to save memory and speed up evaluation
                model_predictions = model.model(**features, return_dict=True)  # Get model predictions
                logits = activation_fnc(model_predictions.logits)  # Apply sigmoid activation

                # Reshape logits if model has a single output label
                if model.config.num_labels == 1:
                    logits = logits.view(-1)

                # Compute the MSE loss
                loss_value = loss_fnc(logits, labels)

            total += 1  # Increment batch counter
            loss_total += loss_value.cpu().item()  # Accumulate loss

        # Compute the final MSE score
        mse = loss_total / total

        # Print the MSE score
        print("MSE: {:.4f} ({}/{})\n".format(mse, loss_total, total))

        # Save evaluation results to a CSV file if output_path is provided
        if output_path is not None and self.write_csv:
            csv_path = os.path.join(output_path, self.csv_file)

            # If the CSV file does not exist, create it and write headers
            if not os.path.isfile(csv_path):
                with open(csv_path, newline='', mode="w", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow(self.csv_headers)
                    writer.writerow([epoch, steps, mse])
            else:
                # Append the results to the existing CSV file
                with open(csv_path, newline='', mode="a", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow([epoch, steps, mse])

        return mse  # Return the computed MSE score


In [12]:
train_batch_size = 16
num_epochs = 8

### Create Training DataLoader:


In [13]:
# We wrap train_samples, which is a list of InputExample, in a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)


### Create Validation DataLoader:
### Initialize the Evaluator:

In [14]:
val_dataloader = DataLoader(train_samples, shuffle=False, batch_size=train_batch_size)

evaluator = MSEAccuracyEvaluator(val_dataloader,show_progress_bar=True , write_csv=True )

In [15]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logger.info(f"Warmup-steps: {warmup_steps}")

In [16]:
# Train the model
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=100,
    warmup_steps=warmup_steps,

)

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/23 [00:00<?, ?it/s]

MSE: 0.1026 (2.3607464272063226/23)



Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

...

Evaluation:   0%|          | 0/23 [00:00<?, ?it/s]

MSE: 0.0025 (0.05843209129790772/23)



Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/23 [00:00<?, ?it/s]

MSE: 0.0025 (0.05751625949909478/23)



In [17]:
model.save('./output/model')

In [18]:
finetune_model = CrossEncoder("./output/model")

In [19]:
sentences = [["What is the publication date of the NVIDIA Corporation Annual Report 2024?", "The publication date of the NVIDIA Corporation Annual Report 2024 is February 21st, 2024."]]
finetune_model.predict(sentences)

array([0.99996316], dtype=float32)

In [21]:
sentences = [["What is the publication date of the NVIDIA Corporation Annual Report 2024?", "The filing dates of the 10-K reports for NVIDIA Corporation in 2004 are May 20th, March 29th, and April 25th."]]
finetune_model.predict(sentences)

array([0.9898353], dtype=float32)

In [20]:
sentences = [["What is the publication date of the NVIDIA Corporation Annual Report 2024?", "GPUs are used to simulate human intelligence, enabling a deeper understanding of the physical world, and are essential for deep learning algorithms due to their parallel processing capabilities, supported by thousands of computing cores."]]
finetune_model.predict(sentences)

array([0.00068506], dtype=float32)

In [ ]:
finetune_model.push_to_hub("rezarahim/bge-reranker-base-finetuned")

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rezarahim/bge-reranker-base-finetuned/commit/6d3ba8d99fa81608ac91b6ad652fdf5468e8e903', commit_message='Upload CrossEncoder', commit_description='', oid='6d3ba8d99fa81608ac91b6ad652fdf5468e8e903', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rezarahim/bge-reranker-base-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='rezarahim/bge-reranker-base-finetuned'), pr_revision=None, pr_num=None)